## <div align="center"> <h2 align="center"> РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ </h1> </div>
## <div align="center"> <h3 align="center"> Факультет физико-математических и естественных наук </h2> </div>
## <div align="center"> <h3 align="center"> Кафедра математического моделирования и искусственного интеллекта </h2> </div>
## <div align="center"> <h3 align="center"> Дисциплина: МОЗИиИБ </h2> </div>
## Отчет по лабораторной работе № 2
## Студент: Евдокимов Максим Михайлович
## Группа: НФИмд-01-24
## Шифры перестоновки

### 1) Маршрутное шифрование

In [1]:
function route_crypt(text::String, route::Char)
    matrix, n = creating_r(text)
    res = r_crypted(matrix, n)
    if route == 'd'
        res = replace(res, '_' => "")
    end
    return res
end

function creating_r(t::String)
    m, rez = split(t, ""), Vector{Vector{String}}(undef, 0)
    dividers, len, d = [5, 6, 7], length(t), 0
    while d == 0
        if len % dividers[1] == 0
            d = 5
        elseif len % dividers[2] == 0
            d = 6
        elseif len % dividers[3] == 0
            d = 7
        else
            push!(m, "_")
            len += 1
        end
    end
    for i in 1:len
        if i%d == 1
            push!(rez, [])
        end
        push!(rez[end], m[i])
    end
    return rez, [div(len, d), d]
end

function r_crypted(m::Vector, n::Vector)
    ec1, ec2= fill("", n[1], n[2]), ""
    for i in 1:n[1]
        for j in 1:n[2]
            if j%2 == 0
                ec1[i, j] = m[i][end-j+1]
            else
                ec1[i, j] = m[i][j]
            end
        end
    end
    for i in 1:n[1]
        for j in 1:n[2]
            if i%2 == 1
                ec2 *= ec1[i, end-j+1]
            else
                ec2 *= ec1[i, j]
            end
        end
    end
    return ec2
end


encoded_r = "Ой что то мы засиделись братцы, Не пора ли нам разгуляться"
#encoded_r = "0123456789abcdef"
result1 = route_crypt(encoded_r, 'e')
println("Кодирование: '", encoded_r, "' => '", result1, "'")
result2 = route_crypt(result1, 'd')
print("Декодирование: '", result1, "' => '", result2, "'")

Кодирование: 'Ой что то мы засиделись братцы, Не пора ли нам разгуляться' => 'тй чОоот  аы змседилбсь ирцтаы  Не,паро аи нлмар зтулягь_яс_'
Декодирование: 'тй чОоот  аы змседилбсь ирцтаы  Не,паро аи нлмар зтулягь_яс_' => 'Ой что то мы засиделись братцы, Не пора ли нам разгуляться'

### 2) Шифрование с помощью решеток

In [219]:
function grid_crypt(text::String, route::Char, k::Vector)
    matrix, n = creating_g(text)
    cd = deepcopy(k)
    if route == 'd'
        cd = 4 .- cd
        res = g_crypted(matrix, n, cd)
        res = replace(res, '_' => "")
    else
        res = g_crypted(matrix, n, cd)
    end
    return res
end

function creating_g(t::String)
    m, rez = split(t, ""), Vector{Vector{String}}(undef, 0)
    dividers, len, d = [4, 6, 8, 10], length(t), 0
    while d == 0
        if len == dividers[1]^2
            d = dividers[1]
        elseif len == dividers[2]^2
            d = dividers[2]
        elseif len == dividers[3]^2
            d = dividers[3]
        elseif len == dividers[4]^2
            d = dividers[4]
        else
            push!(m, "_")
            len += 1
        end
    end
    for i in 1:len
        if i%d == 1
            push!(rez, [])
        end
        push!(rez[end], m[i])
    end
    return rez, d
end

function g_crypted(m::Vector, n::Int, moves::Vector)
    m_cube = fill("_", n, n)
    for i in 1:2:n
        for j in 1:2:n
            temp = circshift([m[i][j], m[i][j+1], m[i+1][j], m[i+1][j+1]], moves[end])
            pop!(moves)
            m_cube[i, j] = temp[1]
            m_cube[i, j+1] = temp[2]
            m_cube[i+1, j] = temp[3]
            m_cube[i+1, j+1] = temp[4]
        end
    end
    return join(join.(eachrow(m_cube)), "")
end

encoded_g = "Cдать лабу до 28 числа включительно"
#encoded_g = "Make a peace of."
key = [3, 2, 1, 1, 2, 2, 2, 3, 3, 2, 3, 2, 1, 0, 1, 3, 2, 1, 1, 3]
result3 = grid_crypt(encoded_g, 'e', key)
println("Кодирование: '", encoded_g, "' => '", result3, "'")
result4 = grid_crypt(result3, 'd', key)
println("Декодирование: '", result3, "' => '", result4, "'")

Кодирование: 'Cдать лабу до 28 числа включительно' => 'длуадьаCтб  ис8лв о а2ч клнюо_елчьит'
Декодирование: 'длуадьаCтб  ис8лв о а2ч клнюо_елчьит' => 'Cдать лабу до 28 числа включительно'


### 3) таблица Вижинера

In [218]:
function Vigener_crypt(text::String, route::Char, k)
    sw, temp = split(text, ""), split(k, "")
    nw, nk = length(sw), length(temp)
    sk = [temp[1+(j%nk)] for j in 0:nw]
    res = g_crypted(sw, sk, nw, route)
    return res
end

function g_crypted(w1::Vector, w2::Vector, n::Int, r::Char)
    alfTable = [circshift([i for i in 'а':'я'], j) for j in 0:-1:-31]
    m = fill("_", n)
    for i in 1:n
        if !(first(w1[i]) in 'а':'я')
            m[i] = w1[i]
        elseif r == 'e'
            xn = Int(first(w1[i])) - Int('а') + 1
            yn = Int(first(w2[i])) - Int('а') + 1
            m[i] = string(alfTable[yn][xn])
        else
            yn = Int(first(w2[i])) - Int('а') + 1
            xn = 0
            for j in 1:32
                if first(w1[i]) == alfTable[yn][j]
                    xn = j
                end
            end
            m[i] = string(alfTable[1][xn])
        end
    end
    return join(m)
end


encoded_v = "кабы не было зимы в городах и селах"
key = "сервер"
result5 = Vigener_crypt(encoded_v, 'e', key)
println("Кодирование: '", encoded_v, "' => '", result5, "'")
result6 = Vigener_crypt(result5, 'd', key)
println("Декодирование: '", result5, "' => '", result6, "'")

Кодирование: 'кабы не было зимы в городах и селах' => 'ыесэ эц сэрю мшоа у урхюхее н вкывъ'
Декодирование: 'ыесэ эц сэрю мшоа у урхюхее н вкывъ' => 'кабы не было зимы в городах и селах'
